In [65]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import io, requests
import openpyxl

In [66]:
NammaMetro_Stations = pd.read_csv("NammaMetro_Stations_Dataset.csv")
NammaMetro_Stations = NammaMetro_Stations[NammaMetro_Stations['code'] != '[TBC]'].reset_index(drop=True)
display(NammaMetro_Stations)

,code,name_eng,name_kan,lines,is_terminus,is_interchange
0,AGPP,Attiguppe,ಅತ್ತಿಗುಪ್ಪೆ,🟣,False,False
1,BTML,BTM Layout,ಬಿ ಟಿ ಎಂ ಬಡಾವಣೆ,🟡,False,False
2,BYPL,Baiyappanahalli,ಬೈಯ್ಯಪ್ಪನಹಳ್ಳಿ,🟣,False,False
3,BSNK,Banashankari,ಬನಶಂಕರಿ,🟢,False,False
4,JTPM,Benniganahalli,ಬೆನ್ನಿಗಾನಹಳ್ಳಿ,🟣,False,False
...,...,...,...,...,...,...
78,VJRH,Vajarahalli,ವಾಜರಹಳ್ಳಿ,🟢,False,False
79,VJN,Vijayanagar,ವಿಜಯನಗರ,🟣,False,False
80,WHTM,Whitefield (Kadugodi),ವೈಟ್ ಫೀಲ್ಡ್ (ಕಾಡುಗೋಡಿ),🟣,True,False
81,PUTH,Yelachenahalli,ಯಲಚೇನಹಳ್ಳಿ,🟢,False,False


In [67]:
df_hourly = pd.read_excel("https://github.com/Vonter/bmrcl-ridership-hourly/raw/8c569cc40f97ac9f5327b1b32432af2cb6cbacc2/raw/station-hourly.xlsx")
display(df_hourly)

,BUSINESS DATE,STATION,00:00 Hrs To 01:00 Hrs,01:00 Hrs To 02:00 Hrs,02:00 Hrs To 03:00 Hrs,03:00 Hrs To 04:00 Hrs,04:00 Hrs To 05:00 Hrs,05:00 Hrs To 06:00 Hrs,06:00 Hrs To 07:00 Hrs,07:00 Hrs To 08:00 Hrs,...,15:00 Hrs To 16:00 Hrs,16:00 Hrs To 17:00 Hrs,17:00 Hrs To 18:00 Hrs,18:00 Hrs To 19:00 Hrs,19:00 Hrs To 20:00 Hrs,20:00 Hrs To 21:00 Hrs,21:00 Hrs To 22:00 Hrs,22:00 Hrs To 23:00 Hrs,23:00 Hrs To Last train,TOTAL
0,2025-08-01,11-Baiyappanahalli,0,0,0,0,19,83,215,918,...,456,497,880,942,923,489,373,134,15,13836
1,2025-08-01,12-SV Road,0,0,0,0,7,39,85,334,...,573,866,1078,1068,856,433,211,132,26,9067
2,2025-08-01,13-Indiranagar,0,0,0,0,4,41,175,621,...,1301,1660,2303,3456,2982,1866,1248,712,161,24471
3,2025-08-01,14-Halasuru,0,0,0,0,5,51,192,570,...,418,387,566,756,660,580,317,151,17,9924
4,2025-08-01,15-Trinity,0,0,0,0,2,17,50,158,...,537,838,1899,3404,2572,1093,421,319,38,13746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1350,2025-08-18,311-Electronic City,0,0,0,0,0,15,29,85,...,207,242,235,261,178,113,95,43,0,2761
1351,2025-08-18,312-Infosys Foundation Konappana Agrahara,0,0,0,0,10,35,129,263,...,346,474,594,574,411,282,165,45,0,5534
1352,2025-08-18,313-Huskur Road,0,0,0,0,0,6,53,115,...,107,171,147,207,128,77,51,37,0,1954
1353,2025-08-18,314-Biocon Hebbagodi,0,0,0,0,0,5,37,83,...,73,93,113,138,80,59,25,15,0,1461


In [68]:
station_name_fixes = {
    'SV Road': 'Swami Vivekananda Road',
    'MG Road': 'Mahatma Gandhi Road',
    'Vidhana Soudha': 'Dr. B. R. Ambedkar Station, Vidhana Soudha',
    'Sir.M. Visveshwaraya': 'Sir.M.Visvesvaraya Stn., Central College',
    'Kempegowda': 'Nadaprabhu Kempegowda Station, Majestic',
    'Bangalore City Station': 'Krantivira Sangolli Rayanna Railway Station',
    'Hosahalli': 'Sri Balagangadharanatha Swamiji Station, Hosahalli',
    'Nayandahalli': 'Pantharapalya–Nayandahalli',
    'Pattangere': 'Pattanagere',
    'Krishnarajapura': 'Krishnarajapura (K.R.Pura)',
    'Seetharampalya': 'Seetharamapalya',
    'Nallur Halli': 'Nallurhalli',
    'Whitefield (Kadugodi)': 'Whitefield (Kadugodi)',
    'Yeshwanthpur Industry': 'Goraguntepalya',
    'Kuvempu Road': 'Mahakavi Kuvempu Road',
    'KR Market': 'Krishna Rajendra Market',
    'RV Road': 'Rashtreeya Vidyalaya Road',
    'Banashankarl': 'Banashankari',
    'JP Nagar': 'Jaya Prakash Nagar',
    'Puttenahalli': 'Yelachenahalli',
    'Thalghattapura': 'Thalaghattapura',
    'Manjunatha Nagar': 'Manjunathanagara',
}

In [69]:
for col in df_hourly.columns.tolist()[2:-1]:
    col_reformatted = col.split()[0][:2]
    df_hourly[col_reformatted] = df_hourly[col]

df_hourly['date'] = pd.to_datetime(df_hourly['BUSINESS DATE'], format='%Y-%m-%d')
df_hourly['station_name'] = df_hourly['STATION'].str.split('-').str[1]
df_hourly['station_code'] = df_hourly['STATION'].str.split('-').str[0]

df_hourly = df_hourly[['date', 'station_code', 'station_name'] + [f"{n:02d}" for n in range(24)] + ['TOTAL']].copy()
df_hourly['total_check'] = df_hourly['TOTAL'] == df_hourly[[f"{n:02d}" for n in range(24)]].sum(axis=1)
df_hourly = df_hourly.loc[:, :'TOTAL'] if df_hourly.loc[:, 'total_check'].isna().sum() == 0 else df_hourly

df_hourly['station_name'] = df_hourly['station_name'].replace(station_name_fixes)

condition = dict(zip(NammaMetro_Stations['name_eng'], NammaMetro_Stations['code']))
df_hourly['station_code'] = df_hourly['station_name'].map(condition)

df_hourly

,date,station_code,station_name,00,01,02,03,04,05,06,...,15,16,17,18,19,20,21,22,23,TOTAL
0,2025-08-01,BYPL,Baiyappanahalli,0,0,0,0,19,83,215,...,456,497,880,942,923,489,373,134,15,13836
1,2025-08-01,SVRD,Swami Vivekananda Road,0,0,0,0,7,39,85,...,573,866,1078,1068,856,433,211,132,26,9067
2,2025-08-01,IDN,Indiranagar,0,0,0,0,4,41,175,...,1301,1660,2303,3456,2982,1866,1248,712,161,24471
3,2025-08-01,HLRU,Halasuru,0,0,0,0,5,51,192,...,418,387,566,756,660,580,317,151,17,9924
4,2025-08-01,TTY,Trinity,0,0,0,0,2,17,50,...,537,838,1899,3404,2572,1093,421,319,38,13746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1350,2025-08-18,ETCT,Electronic City,0,0,0,0,0,15,29,...,207,242,235,261,178,113,95,43,0,2761
1351,2025-08-18,ECTN,Infosys Foundation Konappana Agrahara,0,0,0,0,10,35,129,...,346,474,594,574,411,282,165,45,0,5534
1352,2025-08-18,HSKR,Huskur Road,0,0,0,0,0,6,53,...,107,171,147,207,128,77,51,37,0,1954
1353,2025-08-18,HBGI,Biocon Hebbagodi,0,0,0,0,0,5,37,...,73,93,113,138,80,59,25,15,0,1461


In [70]:
df_hourly[df_hourly['date'] == '2025-08-11']['TOTAL'].sum(axis=0)

np.int64(795066)

`Record Date,Smart Cards,NCMC,Tokens,QR,Group Ticket,Total`

`2025-08-01,425980,25105,200670,180952,157,832864`

`2025-08-02,300912,17623,248287,217745,414,784981`

`2025-08-04,443237,26260,218087,180895,107,868586`

`2025-08-06,475439,29037,213525,179785,185,897971`

`2025-08-11,503837,32198,303165,208382,450,1048032`

`2025-08-12,525613,34320,263412,208766,576,1032687`

`2025-08-14,488569,33052,293707,268138,322,1083788`

`2025-08-15,194028,14440,386683,144182,396,739729`

`2025-08-16,271190,19134,334983,121204,133,746644`

`2025-08-18,485345,34029,270054,228711,532,1018671`

In [108]:
df_pairhourly = pd.read_excel('https://github.com/Vonter/bmrcl-ridership-hourly/raw/8c569cc40f97ac9f5327b1b32432af2cb6cbacc2/raw/stationpair-hourly.xlsx')

station_code_fixes = {
    'BNSK': 'BSNK',
    'BRCS': 'SRCS',
    'BYPH': 'BYPL',
    'CHPK': 'CKPE',
    'CLG': 'CLGA',
    'CLGT': 'CLGA',
    'JAYN': 'JYN',
    'JHLI': 'JLHL',
    'JIDL': 'CKBD',
    'KRMA': 'KRAM',
    'KRPM': 'KRAM',
    'LAGH': 'LBGH',
    'MGRD': 'MAGR',
    'PATC': 'PATG',
    'RVRD': 'RVR',
    'VJNR': 'VJN',
}

df_pairhourly['STATION'] = df_pairhourly['STATION'].map(station_code_fixes).fillna(df_pairhourly['STATION'])

df_pairhourly = df_pairhourly.rename(columns={"BUSINESS DATE": "date", "STATION": "station_code"})
df_pairhourly = df_pairhourly.rename(columns=station_code_fixes)

df_pairhourly['hour'] = df_pairhourly['date'].str.split('-').str[3].str[:2].astype(int)
df_pairhourly['date'] = df_pairhourly['date'].str.split('-').str[0:3].str.join('-')
df_pairhourly['date'] = pd.to_datetime(df_pairhourly['date'], format='%Y-%m-%d')

df_pairhourly['lines'] = df_pairhourly['station_code'].map(NammaMetro_Stations[NammaMetro_Stations['code']!='[TBC]'].set_index('code')['lines']).copy()

df_pairhourly.dropna(subset=['station_code'], inplace=True)
df_pairhourly = df_pairhourly[['date', 'hour', 'station_code', 'lines']+df_pairhourly.columns[2:-2].tolist()]

In [109]:
inspection_date = '2025-08-14'
inspection_hour = 18

pairhourly_filtered = lambda inspection_date, inspection_hour: \
    df_pairhourly[(df_pairhourly['date'] == inspection_date) 
                & (df_pairhourly['hour'] == inspection_hour if inspection_hour >= 0 else True)].copy()
df_pairhourly_filtered = pairhourly_filtered(inspection_date, inspection_hour)

# with pd.option_context("display.max_rows", None, "display.max_columns", None): 
display(df_pairhourly_filtered)

,date,hour,station_code,lines,BYPL,SVRD,IDN,HLRU,TTY,MAGR,...,OFDC,MSRN,CKBR,BSRD,HOSR,ETCT,ECTN,HSKR,HBGI,BMSD
20205,2025-08-14,18,BYPL,🟣,7,15,66,38,17,32,...,1,0,1,4,0,3,5,1,0,4
20206,2025-08-14,18,SVRD,🟣,42,0,36,20,7,20,...,0,3,1,2,1,1,4,1,0,4
20207,2025-08-14,18,IDN,🟣,234,34,11,63,26,89,...,1,2,0,4,0,7,3,1,1,7
20208,2025-08-14,18,HLRU,🟣,81,22,44,6,11,43,...,0,3,1,1,0,0,1,0,1,4
20209,2025-08-14,18,TTY,🟣,220,56,163,128,9,55,...,1,6,0,5,1,1,3,4,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20285,2025-08-14,18,ETCT,🟡,1,1,0,2,0,0,...,18,15,9,31,1,6,9,8,4,11
20286,2025-08-14,18,ECTN,🟡,11,8,3,1,0,1,...,28,37,24,23,5,10,1,14,9,21
20287,2025-08-14,18,HSKR,🟡,1,0,1,0,0,0,...,10,27,7,17,0,2,3,5,1,12
20288,2025-08-14,18,HBGI,🟡,1,1,0,0,0,1,...,10,5,3,16,3,7,9,3,5,9


In [110]:
def inspect_same_in_and_out(df):
    same_in_and_out_df = NammaMetro_Stations.copy()
    same_in_and_out_df['sio_count'] = [0] * len(same_in_and_out_df)
    same_in_and_out_df = same_in_and_out_df.reset_index(drop=True)

    for idx in df.index:
        station = df.loc[idx, 'station_code']
        count = int(df.loc[idx, station])
        same_in_and_out_df.loc[same_in_and_out_df['code']==station, 'sio_count'] += count

    same_in_and_out_df = same_in_and_out_df.set_index('code').T
    same_in_and_out_df = same_in_and_out_df[df.columns.tolist()[4:]]
    same_in_and_out_df = same_in_and_out_df.T.reset_index()

    return same_in_and_out_df

same_in_and_out_df = inspect_same_in_and_out(df_pairhourly_filtered)
with pd.option_context("display.max_rows", None, "display.max_columns", None): 
    display(same_in_and_out_df)

,code,name_eng,name_kan,lines,is_terminus,is_interchange,sio_count
0,BYPL,Baiyappanahalli,ಬೈಯ್ಯಪ್ಪನಹಳ್ಳಿ,🟣,False,False,7
1,SVRD,Swami Vivekananda Road,ಸ್ವಾಮಿ ವಿವೇಕಾನಂದ ರಸ್ತೆ,🟣,False,False,0
2,IDN,Indiranagar,ಇಂದಿರಾ ನಗರ,🟣,False,False,11
3,HLRU,Halasuru,ಹಲಸೂರು,🟣,False,False,6
4,TTY,Trinity,ಟ್ರಿನಿಟಿ,🟣,False,False,9
5,MAGR,Mahatma Gandhi Road,ಮಹಾತ್ಮಾ ಗಾಂಧಿ ರಸ್ತೆ,🟣🌸,False,False,10
6,CBPK,Cubbon Park,ಕಬ್ಬನ್ ಪಾರ್ಕ್,🟣,False,False,0
7,VDSA,"Dr. B. R. Ambedkar Station, Vidhana Soudha","ಡಾ. ಬಿ. ಆರ್. ಅಂಬೇಡ್ಕರ್ ನಿಲ್ದಾಣ, ವಿಧಾನ ಸೌಧ",🟣,False,False,9
8,VSWA,"Sir.M.Visvesvaraya Stn., Central College","ಸರ್ ಎಂ.ವಿಶ್ವೇಶ್ವರಯ್ಯ ನಿಲ್ದಾಣ , ಸೆಂಟ್ರಲ್ ಕಾಲೇಜು",🟣,False,False,9
9,KGWA,"Nadaprabhu Kempegowda Station, Majestic","ನಾಡಪ್ರಭು ಕೆಂಪೇಗೌಡ ನಿಲ್ದಾಣ, ಮೆಜೆಸ್ಟಿಕ್",🟣🟢,False,True,7


In [111]:
print(f"Total sio_count = {same_in_and_out_df['sio_count'].sum()}")

same_in_and_out_df_sorted = same_in_and_out_df.sort_values(by='sio_count', ascending=False)
same_in_and_out_df_sorted = same_in_and_out_df_sorted[same_in_and_out_df_sorted['sio_count'] > 0]
same_in_and_out_df_sorted = same_in_and_out_df_sorted.reset_index(drop=True)

display(same_in_and_out_df_sorted)

Total sio_count = 336


,code,name_eng,name_kan,lines,is_terminus,is_interchange,sio_count
0,JDHP,Jayadeva Hospital,ಜಯದೇವ ಆಸ್ಪತ್ರೆ,🟡🌸,False,True,30
1,YPM,Yeshwanthpur,ಯಶವಂತಪುರ,🟢,False,False,21
2,SRCS,Krantivira Sangolli Rayanna Railway Station,ಕ್ರಾಂತಿವೀರ ಸಂಗೊಳ್ಳಿ ರಾಯಣ್ಣ ರೈಲು ನಿಲ್ದಾಣ,🟣,False,False,19
3,HSRL,Bommanahalli,ಬೊಮ್ಮನಹಳ್ಳಿ,🟡,False,False,12
4,IDN,Indiranagar,ಇಂದಿರಾ ನಗರ,🟣,False,False,11
...,...,...,...,...,...,...,...
61,JYN,Jayanagar,ಜಯನಗರ,🟢,False,False,1
62,MDVP,Singayyanapalya,ಸಿಂಗಯ್ಯನಪಾಳ್ಯ,🟣,False,False,1
63,VDHP,Nallurhalli,ನಲ್ಲೂರುಹಳ್ಳಿ,🟣,False,False,1
64,RRRN,Rajarajeshwari Nagar,ರಾಜರಾಜೇಶ್ವರಿ ನಗರ,🟣,False,False,1


In [ ]:
date_sequence = df_pairhourly['date'].dt.strftime('%Y-%m-%d').unique()
n_rows, n_cols = len(date_sequence), len(NammaMetro_Stations)

selfpairs = []
for inspection_date in date_sequence:
    selfpairs.append(inspect_same_in_and_out(pairhourly_filtered(inspection_date, -1))['sio_count'].values)

selfpairs = np.array(selfpairs)

print(f"selfpairs matrix: {selfpairs.shape[0]} dates x {selfpairs.shape[1]} stations")

selfpairs matrix: 18 dates x 83 stations


In [ ]:
selfpairs_rowsum = np.sum(selfpairs, axis=1)
selfpairs_rowsum_df = pd.DataFrame({"date": date_sequence, "sio_total": selfpairs_rowsum})
selfpairs_rowsum_df

,date,sio_total
0,2025-08-01,1999
1,2025-08-02,2248
2,2025-08-03,2033
3,2025-08-04,2192
4,2025-08-05,1912
5,2025-08-06,2004
6,2025-08-07,1994
7,2025-08-08,1876
8,2025-08-09,2303
9,2025-08-10,2367


In [155]:
selfpairs_colsum = np.sum(selfpairs, axis=0)
selfpairs_colsum_df = pd.DataFrame({"station": df_pairhourly.columns[4:], "sio_total": selfpairs_colsum})
selfpairs_colsum_df.sort_values(by="sio_total", ascending=False)

,station,sio_total
10,SRCS,3371
9,KGWA,2748
24,JTPM,2324
43,YPM,2145
5,MAGR,1816
...,...,...
81,HBGI,166
80,HSKR,155
40,PYID,147
77,HOSR,136


In [169]:
print(f"{np.sum(selfpairs):3,} riders entered and exited through\n\
the same station with a single ticket\n\
between {date_sequence[0]} and {date_sequence[-1]}.")

48,280 riders entered and exited through
the same station with a single ticket
between 2025-08-01 and 2025-08-18.
